<a href="https://colab.research.google.com/github/AlexSan1910/Consumo_API/blob/main/Consultando_CNPJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A intenção dessa automação é buscar duas bases de dados, onde uma base de dados possui informações que já existem no banco de uma empresa, e a outra base são informações misturadas. Objetivo principal é buscar o CNPJ dessas duas listas, fazer um depara entre elas, e buscar apenas os cnpj que não esão no banco da empresa e consultar todas as informações dela em uma API.

Com esses dados, seria possivel identificar potenciais clientes para a empresa. Porem para todo esse processo acontecer, existe diversas etapas que devem ser percorridas para tratamento dessas duas bases, e no final salvar em uma terceira base de dados no google drive, para posteriormente se conectar com um power B.I e a partir do relatorio ter algumas analises sobre protenciais estabelecimentos.

**INSTALANDO PACOTES NECESSARIOS LOCALMENTE NA MAQUINA VIRTUAL**

In [ ]:
!pip install selenium
!pip install webdriver_manager
!pip install pandas
!pip install requests
!pip install json
!pip install time
!pip install os
!pip install pyspark

**INSTALANDO TODAS AS BIBLIOTECAS NECESSARIAS PARA O FUNCIONAMENTO DO CODIGO**

In [2]:
# importação bibliotecas
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from datetime import datetime, timedelta
import os
import sys
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
import numpy as np
import math
import re
import datetime
from google.colab import drive
import requests
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
import openpyxl

**ENTRANDO NO GOOGLE DRIVE, BUSCANDO CADA UM DOS ARQUIVOS E DEFININDO SEUS CAMINHOS**

In [ ]:
# Autenticação com o google drive quando necessario
drive.mount('/content/drive')

# Caminhos dos arquivos no Google Drive
caminho_infox = "/content/drive/MyDrive/base_dados/infox.xlsx"
caminho_adq = "/content/drive/MyDrive/base_dados/adq.xlsx"

# Criando o objeto SparkSession
spark = SparkSession.builder.appName("Exemplo DataFrame").getOrCreate()

# Ler os arquivos XLSX do Google Drive
df_infox = pd.read_excel(caminho_infox)
df_adq = pd.read_excel(caminho_adq)

# **EFETUANDO TRATAMENTO DOS DADOS**

Todo lugar que possui valor nullo, será adicionado 0




In [ ]:
# Substituindo todos os valores nulos por 0. pois para mudar o tipo primitivo da coluna CNPJ por inteiro é necessario retirar todos os valores nulos
df_infox.fillna(0, inplace=True)
df_adq.fillna(0, inplace=True)

In [ ]:
# Criar as listas de CNPJs com a coluna CNPJ de cada um dos arquivos
lista_infox = df_infox['CNPJ'].tolist()
lista_adq = df_adq['CNPJ'].tolist()

# mostrando os valores distintos da coluna CNPJ de cada um dos df
print("Valores Unicos:")
print(df_adq["CNPJ"].nunique())
print(df_infox["CNPJ"].nunique())

print("\nExemplos de cada lista:")
print(lista_infox[:5])
print(lista_adq[:5])

adicionando em um dataframe apenas os cnpj das duas base de dados, tanto do crm
quanto das adquirentes


In [ ]:
# criando função que transforma as listas em strings
def listaParaString(lista):
  lista_tratados = []
  for cnpj in lista:
    cnpj = str(cnpj)
    lista_tratados.append(cnpj)

  return lista_tratados

lista_infox = listaParaString(lista_infox)
lista_adq = listaParaString(lista_adq)

print(lista_infox[:5])
print(lista_adq[:5])

In [ ]:
# Criando função que faz padronização dos cnpj tirando ".0"
# Para que cada um dos cnpj's possam se transformar em int, porque do jeito que estava antes estava como float
def padronizandoCnpj(lista):
  lista_tratados = []
  for cnpj in lista:
    cnpj = cnpj[:-2]
    lista_tratados.append(cnpj)

  return lista_tratados

lista_adq = padronizandoCnpj(lista_adq)

print(lista_infox[:5])
print(lista_adq[:5])

**Preenchendo os CNPJs com zeros à esquerda para garantir 14 dígitos. Da lista da Mega**

In [ ]:
# Criando função que completa com zeros a esquerda os cnpj o suficiente até completar 14 digitos
def completandoCnpj(lista):
  lista_tratados = []
  for cnpj in lista:
    cnpj = str(cnpj).zfill(14) # Adiciona zeros á esquerda até completar 14 digitos
    lista_tratados.append(cnpj)

  return lista_tratados

lista_infox = completandoCnpj(lista_infox)
lista_adq = completandoCnpj(lista_adq)

print(lista_infox[:5])
print(lista_adq[:5])

Fazendo depara de quais cnpj existe na base de dados das adquirentes que não existe na base de dados do CRM e colocando em um dataframe somente os CNPJs que não estão nessa base de dados

In [ ]:
# Encontrar os CNPJs da listaAdq que não estão na listaInfox e colocar dentro da variavel cnpjs_nao_encontrados
def comparacaoListas(lista1, lista2):
  lista_tratada = list(set(lista1) - set(lista2))

  return lista_tratada

cnpjs_nao_encontrados = comparacaoListas(lista_adq, lista_infox)

In [ ]:
# função para encontrar valores unicos de uma lista
def valoresUnicos(lista):
  lista_tratada = []
  for item in lista:
    if item not in lista_tratada:
      lista_tratada.append(item)
  return lista_tratada


print(f"Quantidade da lista: {len(cnpjs_nao_encontrados)}")
print(f"Valores Unicos: {len(valoresUnicos(cnpjs_nao_encontrados))}")
print(f"Lista: {cnpjs_nao_encontrados}")

Criando uma estrutura de repetição, no qual irá fazer o consumo de cada cnpj na API da Machine System, que trará todas as informações do CNPJ e irá colocar no em uma planilha no excel.

In [ ]:
vetor_cnpj_nao_encontrados = []
# Loop para consultar os CNPJs e salvar os dados no DataFrame
for cnpj in cnpjs_nao_encontrados:
    try:
      # caminho para colocar o dados tratados
      caminho_excel_saida = '/content/drive/MyDrive/base_dados/base_tratada.xlsx'

      # criando um dataframe com os dados da url acima em python
      df_saida = pd.read_excel(caminho_excel_saida)

      # lendo o dataframe do pandas no spark
      df_saida_spark = spark.createDataFrame(df_saida)

      time.sleep(20) # Aguardando 10 segundos para cada requisição

      # Dados para a chamada cURL
      token_api = "token"

      # URL base
      base_url = 'http://104.251.212.117/cnpj/validador.php'

      # Parâmetros da requisição
      data = {
          'cnpj': cnpj,
          'token': token_api
      }

      # Headers de aceitação para JSON
      headers = {
        'Accept': 'application/json',
      }
      # Realiza a requisição POST com headers de aceitação
      resposta = requests.post(base_url, data=data, headers=headers).json()

      cnpj_empresa = cnpj
      data_abertura = resposta["abertura"]
      situacao_empresa = resposta["situacao"]
      tipo_empresa = resposta["tipo"]
      nome_fantasia = resposta["nome"]
      porte_empresa = resposta["porte"]
      natureza_juridica = resposta["natureza_juridica"]
      codigo_cnae_principal = resposta["atividade_principal"][0]["code"]
      nome_cnae_principal = resposta["atividade_principal"][0]["text"]
      codigo_cnae_secundario = resposta["atividades_secundarias"][0]["code"]
      nome_cnae_secundario = resposta["atividades_secundarias"][0]["text"]
      logradouro = resposta["logradouro"]
      numero = resposta["numero"]
      complemento = resposta["complemento"]
      municipio = resposta["municipio"]
      bairro = resposta["bairro"]
      uf = resposta["uf"]
      cep = resposta["cep"]
      email = resposta["email"]
      telefone = resposta["telefone"]
      data_situacao = resposta['data_situacao']
      ultima_atualizacao = resposta['ultima_atualizacao']
      status = resposta['status']
      fantasia = resposta['fantasia']
      efr = resposta['efr']
      motivo_situacao = resposta['motivo_situacao']
      situacao_especial = resposta['situacao_especial']
      data_situacao_especial = resposta['data_situacao_especial']
      capital_social = resposta['capital_social']
      extra = resposta['extra']

      # printar todas as variaveis acima
      print(f'''
          CNPJ {cnpj_empresa} Existente!
      ''')

      # Criando base de dados para o novo dataframe que será criado
      novos_dados = [
          {
              "cnpj_empresa": cnpj_empresa,
              "data_abertura": data_abertura,
              "situacao_empresa": situacao_empresa,
              "tipo_empresa": tipo_empresa,
              "nome_fantasia": nome_fantasia,
              "porte_empresa": porte_empresa,
              "natureza_juridica": natureza_juridica,
              "codigo_cnae_principal": codigo_cnae_principal,
              "nome_cnae_principal": nome_cnae_principal,
              "codigo_cnae_secundario": codigo_cnae_secundario,
              "nome_cnae_secundario": nome_cnae_secundario,
              "logradouro": logradouro,
              "numero": numero,
              "complemento": complemento,
              "municipio": municipio,
              "bairro": bairro,
              "uf": uf,
              "cep": cep,
              "email": email,
              "telefone": telefone,
              "data_situacao": data_situacao,
              "ultima_atualizacao": ultima_atualizacao,
              "status": status,
              "fantasia": fantasia,
              "efr": efr,
              "motivo_situacao": motivo_situacao,
              "situacao_especial": situacao_especial,
              "data_situacao_especial": data_situacao_especial,
              "capital_social": capital_social,
              "extra": extra
          }
      ]

      # definindo o tipo de cada um dos dados da base dados anterior
      esquema = StructType([

          StructField("cnpj_empresa", StringType(), True),
          StructField("data_abertura", StringType(), True),
          StructField("situacao_empresa", StringType(), True),
          StructField("tipo_empresa", StringType(), True),
          StructField("nome_fantasia", StringType(), True),
          StructField("porte_empresa", StringType(), True),
          StructField("natureza_juridica", StringType(), True),
          StructField("codigo_cnae_principal", StringType(), True),
          StructField("nome_cnae_principal", StringType(), True),
          StructField("codigo_cnae_secundario", StringType(), True),
          StructField("nome_cnae_secundario", StringType(), True),
          StructField("logradouro", StringType(), True),
          StructField("numero", StringType(), True),
          StructField("complemento", StringType(), True),
          StructField("municipio", StringType(), True),
          StructField("bairro", StringType(), True),
          StructField("uf", StringType(), True),
          StructField("cep", StringType(), True),
          StructField("email", StringType(), True),
          StructField("telefone", StringType(), True),
          StructField("data_situacao", StringType(), True),
          StructField("ultima_atualizacao", StringType(), True),
          StructField("status", StringType(), True),
          StructField("fantasia", StringType(), True),
          StructField("efr", StringType(), True),
          StructField("motivo_situacao", StringType(), True),
          StructField("situacao_especial", StringType(), True),
          StructField("data_situacao_especial", StringType(), True),
          StructField("capital_social", StringType(), True),
          StructField("extra", StringType(), True)
      ])

      # criando um novo dataframe com os dados que foi resgatado na API
      df_novo = spark.createDataFrame(novos_dados, schema=esquema)

      # unindo os dois dataframe, o atual que já existia com o novo criado agora
      df_unido = df_saida_spark.union(df_novo)

      # Convertendo o dataframe do spark para pandas
      df_pandas = df_unido.toPandas()

      # Salvando os dados no caminho dos dados tratados
      df_pandas.to_excel(caminho_excel_saida, index=False)

    except:
      print(f'CNPJ: {cnpj} NÃO EXISTE!')
      vetor_cnpj_nao_encontrados.append(cnpj)

print(vetor_cnpj_nao_encontrados)
print(f"A quantidade de CNPJ não encontrados foi de {len(vetor_cnpj_nao_encontrados)} de {len(cnpjs_nao_encontrados)}")